In [1]:
#!pip install fake_useragent

In [2]:
import pandas as pd
import numpy as np
import requests
import time
from bs4 import BeautifulSoup
from fake_useragent import UserAgent
from tqdm import tqdm_notebook

In [3]:
# подгрузим один из методов этой библиотеки
from fake_useragent import UserAgent

In [4]:
UserAgent().chrome

'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/113.0.0.0 Safari/537.36'

## Описание задачи

Парсинг сайта https://topliba.com/ с целью сбора данных (рейтинг, название фильма, описание, жанр, год выпуска) по жанру Фэнтези. Дело в том, что большинство современных книжный сайтов не предоставляют возможности делать сквозной  поиск по всем страницам, а только в рамках делений по жанрам. Наиболее представленный жанр н данном сайте - фэнтези (783 страницы поиска) по нему и будем делать парсинг. На данном сайте список фильмов подгружается динамически по страницам. Обратимся к странице Фэнтези, и запишем запрос к данной странице.

In [5]:
page_link = 'https://topliba.com/genres/12090?p=1'

In [6]:
response = requests.get(page_link, headers={'User-Agent': UserAgent().chrome})
response

<Response [200]>

In [7]:
html = response.content

### Получение ссылок с фильмами

С помощью библиотеки BeautifulSoup распарсим страницу и найдем ссылки к фильмам. 
Ссылки к фильмам находятся в теге div с классом discovery-grids-container.

In [8]:
soup = BeautifulSoup(response.content, 'html.parser')

In [9]:
response = requests.get(page_link, headers={'User-Agent': UserAgent().chrome})
html = response.content
html[:1000]

b'<!DOCTYPE html>\n<html lang="ru">\n    <head>\n        <meta charset="utf-8">\n        <meta http-equiv="X-UA-Compatible" content="IE=edge">\n        <meta name="viewport" content="width=device-width, initial-scale=1">\n        <title>\xd0\xa4\xd1\x8d\xd0\xbd\xd1\x82\xd0\xb5\xd0\xb7\xd0\xb8: \xd1\x81\xd0\xba\xd0\xb0\xd1\x87\xd0\xb0\xd1\x82\xd1\x8c \xd0\xba\xd0\xbd\xd0\xb8\xd0\xb3\xd0\xb8 \xd0\xb2 fb2, \xd1\x87\xd0\xb8\xd1\x82\xd0\xb0\xd1\x82\xd1\x8c \xd0\xbe\xd0\xbd\xd0\xbb\xd0\xb0\xd0\xb9\xd0\xbd \xe2\x80\xa2 \xd0\xa1\xd1\x82\xd1\x80\xd0\xb0\xd0\xbd\xd0\xb8\xd1\x86\xd0\xb0 1</title>\n        <meta name="description" content="\xd0\x9a\xd0\xbd\xd0\xb8\xd0\xb3\xd0\xb8 \xd0\xb6\xd0\xb0\xd0\xbd\xd1\x80\xd0\xb0 \xd0\xa4\xd1\x8d\xd0\xbd\xd1\x82\xd0\xb5\xd0\xb7\xd0\xb8 \xd0\xb4\xd0\xbe\xd1\x81\xd1\x82\xd1\x83\xd0\xbf\xd0\xbd\xd1\x8b\xd0\xb5 \xd0\xb2 \xd1\x84\xd0\xbe\xd1\x80\xd0\xbc\xd0\xb0\xd1\x82\xd0\xb5 fb2, \xd0\xb0 \xd1\x82\xd0\xb0\xd0\xba\xd0\xb6\xd0\xb5 \xd0\xb4\xd0\xbb\xd1\x8f \xd1\x

In [10]:
book_container = soup.findAll(lambda tag: tag.name == 'a' and tag.get('class') == ['book-title'])
book_container[:5]

[<a class="book-title" href="https://topliba.com/books/898208" title="Скачать книгу Земная">Земная</a>,
 <a class="book-title" href="https://topliba.com/books/898197" title="Скачать книгу Проклятая">Проклятая</a>,
 <a class="book-title" href="https://topliba.com/books/898032" title="Скачать книгу Этот лот (не) продается!!!">Этот лот (не) продается!!!</a>,
 <a class="book-title" href="https://topliba.com/books/897776" title="Скачать книгу Дитя солнца">Дитя солнца</a>,
 <a class="book-title" href="https://topliba.com/books/898031" title="Скачать книгу Проклятье чёрной луны">Проклятье чёрной луны</a>]

In [11]:
# Cписок ссылок на каждую книгу со страницы
book_links = [link.attrs['href'] for link in 
                  soup.findAll(lambda tag: tag.name == 'a' and tag.get('class') == ['book-title'])]
book_links[:5]

['https://topliba.com/books/898208',
 'https://topliba.com/books/898197',
 'https://topliba.com/books/898032',
 'https://topliba.com/books/897776',
 'https://topliba.com/books/898031']

На сайте https://topliba.com/ book_links уже выводятся полностью, вместе с названием сайта, поэтому вместо ['https://topliba.com' + link.attrs['href'] for link in book_links] будем подставлять только ['link.attrs['href'] for link in book_links]

In [12]:
def getPageLinks(page_number):
    """
        Возвращает список ссылок на книги, полученный с текущей страницы
        
        Параметры:
            page_number: int/string - номер страницы для парсинга            
    """
    # составляем ссылку на страницу поиска
    page_link = 'https://topliba.com/genres/12090?p={}'.format(page_number)
    
    # запрашиваем данные о ней
    response = requests.get(page_link, headers={'User-Agent': UserAgent().chrome})
        
    if not response.ok:
        # если сервер нам отказал, вернем пустой лист для текущей страницы
        print('Сервер ничего не вернул')
        return [] 
    
    # получаем содержимое страницы и переводим в суп
    soup = BeautifulSoup(response.content, 'html.parser')
    
    # ищем ссылки на книги и формируем полные ссылки
    book_links = soup.findAll(lambda tag: tag.name == 'a' and tag.get('class') == ['book-title'])
    book_links = [link.attrs['href'] for link in book_links]
    
    return book_links

In [13]:
book_links = getPageLinks(3)
book_links

['https://topliba.com/books/893175',
 'https://topliba.com/books/892994',
 'https://topliba.com/books/891273',
 'https://topliba.com/books/891793',
 'https://topliba.com/books/891792',
 'https://topliba.com/books/891095',
 'https://topliba.com/books/891044',
 'https://topliba.com/books/890657',
 'https://topliba.com/books/890813',
 'https://topliba.com/books/889932',
 'https://topliba.com/books/889824',
 'https://topliba.com/books/889772',
 'https://topliba.com/books/889771',
 'https://topliba.com/books/883986',
 'https://topliba.com/books/887325',
 'https://topliba.com/books/883928',
 'https://topliba.com/books/889619',
 'https://topliba.com/books/889141',
 'https://topliba.com/books/889278',
 'https://topliba.com/books/889169',
 'https://topliba.com/books/888662',
 'https://topliba.com/books/888765',
 'https://topliba.com/books/888636',
 'https://topliba.com/books/888704']

### Получим информации о книге
Обратимся к странице с фильмом Сердце Дракона. Том 2 и достанем следующую информацию:
- Название книги.
- Автор книги
- Оценка книги
- Аннотация/Описание
- Количество читателей
- Количество комментариев/отзывов

In [14]:
book_page = 'https://topliba.com/books/736796'
response = requests.get(book_page, headers={'User-Agent': UserAgent().chrome})

html = response.content
soup = BeautifulSoup(response.content, 'html.parser')

### Получим название книги

In [15]:
book_title = soup.find(lambda tag: tag.name == 'div' and tag.get('class') == ['row'])
book_title

<div class="row">
<div class="header-background"><div style="background: url(https://topliba.com/covers/736796.jpg?t=1705317373)"></div></div>
<div class="col-xs-12 col-sm-3 cover">
<div itemscope="" itemtype="http://schema.org/ImageObject" style="position: relative">
<meta content="Лисина Александра - Щит императора" itemprop="name">
<img alt="Лисина Александра - Щит императора" itemprop="contentUrl" src="https://topliba.com/covers/736796.jpg?t=1705317373" title="Лисина Александра - Щит императора"/>
</meta></div>
</div>
<div class="col-xs-12 col-sm-9 book-title">
<h1 itemprop="name">Щит императора</h1>
<h2 class="book-author" itemprop="author">
<a href="https://topliba.com/authors/2142" title="Книги Александра Лисина">Александра Лисина</a>
</h2>
</div>
</div>

Заголовок книги находится в теге h1, который имеет атрибут itemprop="name". 

In [16]:
book_title = soup.find('h1', itemprop='name').text
book_title

'Щит императора'

In [17]:
#book_title = soup.find(lambda tag: tag.name == 'h1') # and tag.get('itemprop') == ['name'])
#book_title = film_title.text
#book_title

### Получим автора книги

In [18]:
book_author = soup.find(lambda tag: tag.name == 'h2' and tag.get('class') == ['book-author'])
book_author = book_author.text.replace("\n", "")
book_author

'Александра Лисина'

### Получим аннотацию к книге

In [19]:
book_announs = soup.find(lambda tag: tag.name == 'div' and tag.get('class') == ['description']).text
book_announs = book_announs.strip()
book_announs

'После смерти великого императора Орриана на престол вступает его единственный сын, которому в наследство достается не только страна, но и телохранитель. Карриану не нужен такой довесок к короне, однако нарушить волю отца он не рискует. Мару тоже не нравится это решение, но приказ хозяина для него превыше собственных предпочтений. Для двух врагов подобное соседство — худшее, что только можно измыслить. Тень, не желающая оберегать нового императора, и император, ненавидящий свою тень… Казалось бы, у этих двоих нет и не может быть ничего общего. Но жизнь все расставляет по своим местам.'

### Получим рейтинг книги 

Рейтинги книг хранятся в аттрибутах тега div class="col-lg-12". Посмотрим на структуру данного тега.

In [20]:
scoreboard = soup.find(lambda tag: tag.name == 'div' and tag.get('class') == ['col-lg-12'])
scoreboard

<div class="col-lg-12">
<rating _avgrating="4.75" _rating="0" _ratingsnumber="251" _showavgrating="true" _showavgratingtext="true" _url="/books/736796/ratings" _user="" class="pull-left"></rating>
<meta content="1" itemprop="worstRating"/>
<meta content="5" itemprop="bestRating"/>
<meta content="251" itemprop="ratingCount"/>
<meta content="4.75" itemprop="ratingValue"/>
</div>

In [21]:
def getRating(soup, rate):
    try:
        obj = soup.find('rating', attrs={'class':'pull-left'})
        obj = obj.get(rate)
    except:
        obj=None
    
    return obj

In [22]:
getRating(soup, '_avgrating')

'4.75'

### Получим количество читателей книги и количества комментариев (отзывов)

Сведения о читателях хранятся в book-readers

In [23]:
book_readers = soup.find(lambda tag: tag.name == 'div' and tag.get('class') == ['book-readers'])
book_readers

<div class="book-readers">
<div class="col-lg-8 col-sm-12 well-sm">
<div class="col-sm-2 col-xs-4 book-readers-status">
<span class="book-readers-status-count">490</span>
<span>читателей</span>
</div>
<div class="col-sm-2 col-xs-4 book-readers-status">
<a href="https://topliba.com/books/736796/impressions">
<span class="book-readers-status-count">88</span>
<span>отзывов</span>
</a>
</div>
<div class="col-sm-2 col-xs-4 book-readers-status">
<a href="https://topliba.com/books/736796/quotes">
<span class="book-readers-status-count">20</span>
<span>цитат</span>
</a>
</div>
</div>
</div>

In [24]:
# Находим все span элементы с классом 'book-readers-status-count'
readers_count_elements = soup.find_all('span', class_='book-readers-status-count')

film_readership_count = readers_count_elements[0].string  
film_comment_count = readers_count_elements[1].string  

print("Количество читателей:", film_readership_count)
print("Количество отзывов:", film_comment_count)

Количество читателей: 490
Количество отзывов: 88


### Получение данных о жанре и годе издания

In [25]:
# Находим все теги div с классом 'col-lg-12'
search_elements = soup.find_all('div', class_='col-lg-12')
search_elements

[<div class="col-lg-12">
 <rating _avgrating="4.75" _rating="0" _ratingsnumber="251" _showavgrating="true" _showavgratingtext="true" _url="/books/736796/ratings" _user="" class="pull-left"></rating>
 <meta content="1" itemprop="worstRating"/>
 <meta content="5" itemprop="bestRating"/>
 <meta content="251" itemprop="ratingCount"/>
 <meta content="4.75" itemprop="ratingValue"/>
 </div>,
 <div class="col-lg-12">
 <a href="https://topliba.com/genres/12090" title="Книги жанра Фэнтези">Фэнтези</a>, <a href="https://topliba.com/genres/12147" title="Книги жанра Попаданцы">Попаданцы</a>
 </div>,
 <div class="col-lg-12">
         Год выхода: <a href="https://topliba.com/years/2020" title="Книги 2020 года">2020</a>
 </div>,
 <div class="col-lg-12">
             Серия: <a href="https://topliba.com/series/267" title='Книги серии "Магия фэнтези"'>Магия фэнтези</a>, <a href="https://topliba.com/series/370269" title='Книги серии "Слуга императора"'>Слуга императора</a> (#2)
         </div>,
 <div clas

In [26]:
"""
genre = search_elements[1].text.replace('\n', '')  
year = search_elements[2].text.replace('\n', '')

print("Жанры:", genre)
print("Год:", year)
"""

'\ngenre = search_elements[1].text.replace(\'\n\', \'\')  \nyear = search_elements[2].text.replace(\'\n\', \'\')\n\nprint("Жанры:", genre)\nprint("Год:", year)\n'

Запишем итоговую функцию для сбора данных по книге.

In [27]:
def getBookData(book_page):
    book_data = pd.DataFrame(columns=['Title', 'Author', 'Description', 'Raiting', 'Readers', 'Comments'], index=[0])
    
    response = requests.get(book_page, headers={'User-Agent': UserAgent().chrome})
    
    if not response.ok:
        # если сервер нам отказал, вернем статус ошибки 
        return response.status_code
    
    # получаем содержимое страницы и переводим в суп
    soup = BeautifulSoup(response.content, 'html.parser')
     
    book_title = soup.find('h1', itemprop='name').text
    book_data['Title'] = book_title  
                
    book_author = soup.find(lambda tag: tag.name == 'h2' and tag.get('class') == ['book-author'])
    book_author = book_author.text.replace("\n", "")
    book_data['Author'] = book_author
    
    book_announs = soup.find(lambda tag: tag.name == 'div' and tag.get('class') == ['description']).text
    book_announs = book_announs.replace('\n', '')
    book_announs = book_announs.strip()
    book_data['Description']= book_announs
      
    book_data['Raiting'] = getRating(soup=soup, rate='_avgrating')
    
    readers_count_elements = soup.find_all('span', class_='book-readers-status-count')

    book_readership_count = readers_count_elements[0].string  
    book_data['Readers'] = book_readership_count
    
    book_comment_count = readers_count_elements[1].string  
    book_data['Comments'] = book_comment_count
    
    #search_elements = soup.find_all('div', class_='col-lg-12')
    #genre = search_elements[1].text
    #book_data['Genre']=genre
    
    return book_data

In [28]:
getBookData('https://topliba.com/books/736796')

,Title,Author,Description,Raiting,Readers,Comments
0,Щит императора,Александра Лисина,После смерти великого императора Орриана на пр...,4.75,490,88


In [29]:
"""
def getBookData(film_page):
    book_data = pd.DataFrame(columns=['Title', 'Author', 'Description', 'Raiting', 'Readers', 'Comments'], index=[0])
    
    response = requests.get(book_page, headers={'User-Agent': UserAgent().chrome})
    
    if not response.ok:
        return response.status_code
    
    soup = BeautifulSoup(response.content, 'html.parser')
    
    book_title = soup.find('h1', itemprop='name').text
    book_data.at[0, 'Title'] = film_title  # Измененная строка
    
    film_author = soup.find('h2', class_='book-author').text.replace("\n", "")
    film_data.at[0, 'Author'] = film_author
    
    book_announs = soup.find('div', class_='description').text.replace('\n', '').strip()
    book_data.at[0, 'Description'] = book_announs
    
    book_data.at[0, 'Raiting'] = getRating(soup=soup, rate='_avgrating')
    
    readers_count_elements = soup.find_all('span', class_='book-readers-status-count')

    book_readership_count = readers_count_elements[0].string  
    book_data.at[0, 'Readers'] = book_readership_count
    
    book_comment_count = readers_count_elements[1].string  
    book_data.at[0, 'Comments'] = book_comment_count
         
    return book_data

getBookData('https://topliba.com/books/736796')
"""

'\ndef getBookData(film_page):\n    book_data = pd.DataFrame(columns=[\'Title\', \'Author\', \'Description\', \'Raiting\', \'Readers\', \'Comments\'], index=[0])\n    \n    response = requests.get(book_page, headers={\'User-Agent\': UserAgent().chrome})\n    \n    if not response.ok:\n        return response.status_code\n    \n    soup = BeautifulSoup(response.content, \'html.parser\')\n    \n    book_title = soup.find(\'h1\', itemprop=\'name\').text\n    book_data.at[0, \'Title\'] = film_title  # Измененная строка\n    \n    film_author = soup.find(\'h2\', class_=\'book-author\').text.replace("\n", "")\n    film_data.at[0, \'Author\'] = film_author\n    \n    book_announs = soup.find(\'div\', class_=\'description\').text.replace(\'\n\', \'\').strip()\n    book_data.at[0, \'Description\'] = book_announs\n    \n    book_data.at[0, \'Raiting\'] = getRating(soup=soup, rate=\'_avgrating\')\n    \n    readers_count_elements = soup.find_all(\'span\', class_=\'book-readers-status-count\')\n\n

## Получение информации по всем книгам

Соберем информацию по всем книгам. Cоздадим пустой датафрейм, в который мы будем добавлять информацию по каждой книге.

In [30]:
final_df = pd.DataFrame(columns=['Title', 'Author', 'Description', 'Raiting', 'Readers', 'Comments'])

In [31]:
final_df.head()

,Title,Author,Description,Raiting,Readers,Comments


Проверка работы getPageLinks()

In [32]:
def getPageLinks(max_pages):
    # Функция для получения ссылок на страницы с сайта
    # Здесь предполагается, что getPageLinks(784) будет возвращать список ссылок
    
    # Пример кода для получения ссылок на страницы
    page_links = [f'https://topliba.com/genres/12090?p={i}' for i in range(1, max_pages + 1)]
    return page_links

# Теперь вызовем функцию и выведем результат
links = getPageLinks(784)  # Получим ссылки на все страницы
print(links)

['https://topliba.com/genres/12090?p=1', 'https://topliba.com/genres/12090?p=2', 'https://topliba.com/genres/12090?p=3', 'https://topliba.com/genres/12090?p=4', 'https://topliba.com/genres/12090?p=5', 'https://topliba.com/genres/12090?p=6', 'https://topliba.com/genres/12090?p=7', 'https://topliba.com/genres/12090?p=8', 'https://topliba.com/genres/12090?p=9', 'https://topliba.com/genres/12090?p=10', 'https://topliba.com/genres/12090?p=11', 'https://topliba.com/genres/12090?p=12', 'https://topliba.com/genres/12090?p=13', 'https://topliba.com/genres/12090?p=14', 'https://topliba.com/genres/12090?p=15', 'https://topliba.com/genres/12090?p=16', 'https://topliba.com/genres/12090?p=17', 'https://topliba.com/genres/12090?p=18', 'https://topliba.com/genres/12090?p=19', 'https://topliba.com/genres/12090?p=20', 'https://topliba.com/genres/12090?p=21', 'https://topliba.com/genres/12090?p=22', 'https://topliba.com/genres/12090?p=23', 'https://topliba.com/genres/12090?p=24', 'https://topliba.com/gen

In [33]:
book_links = getPageLinks(1000) #  функция для получения ссылок на фильмы

In [34]:
from tqdm import tqdm_notebook

# Первый cпособ организации цикла сбора информации по всем книгам

In [36]:
final_df.head()

,Title,Author,Description,Raiting,Readers,Comments


In [37]:
for book_link in tqdm_notebook(book_links):
    try: 
        book_data = getBookData(book_link)
        final_df = pd.concat((final_df, book_data), axis=0)
        time.sleep(3)
    except:
        continue

C:\Users\Olga\AppData\Local\Temp\ipykernel_11408\2439003353.py:1: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for book_link in tqdm_notebook(book_links):


  0%|          | 0/1000 [00:00<?, ?it/s]

In [38]:
final_df.head()

,Title,Author,Description,Raiting,Readers,Comments


# Второй способ создания цикла - по лекции Марии Тихоновой.

In [ ]:
def getBookData(book_page):
    # запрашиваем данные по ссылке
    response = requests.get(book_page, headers={'User-Agent': UserAgent().chrome})

    if not response.ok:
        # если сервер нам отказал, вернем статус ошибки
        return response.status_code

    # получаем содержимое страницы и переводим в суп
    html = response.content
    soup = BeautifulSoup(html,'html.parser')

    # используя ранее написанные функции парсим информацию
    book_title = soup.find('h1', itemprop='name').text
                 
    book_author = soup.find(lambda tag: tag.name == 'h2' and tag.get('class') == ['book-author'])
    book_author = book_author.text.replace("\n", "")
    
    book_announs = soup.find(lambda tag: tag.name == 'div' and tag.get('class') == ['description']).text
    book_announs = book_announs.replace('\n', '')
    book_announs = book_announs.strip()
         
    raiting = getRating(soup=soup, rate='_avgrating')
    
    readers_count_elements = soup.find_all('span', class_='book-readers-status-count')

    book_readership_count = readers_count_elements[0].string  
        
    book_comment_count = readers_count_elements[1].string  
            
    # составляем словарь, в котором будут хранится все полученные и обработанные данные
    data_row = {'Title':book_title, 'Author':book_author,
                'Description':book_announs, 'Raiting':raiting, 'Readers':book_readership_count,'Comments':book_comment_count}
    return data_row

In [50]:
data_row = getBookData('https://topliba.com/books/736796')

In [51]:
data_row

{'Title': 'Щит императора',
 'Author': 'Александра Лисина',
 'Description': 'После смерти великого императора Орриана на престол вступает его единственный сын, которому в наследство достается не только страна, но и телохранитель. Карриану не нужен такой довесок к короне, однако нарушить волю отца он не рискует. Мару тоже не нравится это решение, но приказ хозяина для него превыше собственных предпочтений. Для двух врагов подобное соседство — худшее, что только можно измыслить. Тень, не желающая оберегать нового императора, и император, ненавидящий свою тень… Казалось бы, у этих двоих нет и не может быть ничего общего. Но жизнь все расставляет по своим местам.',
 'Raiting': '4.75',
 'Readers': '490',
 'Comments': '88'}

In [52]:
final_df = pd.DataFrame(columns=['Title', 'Author', 'Description', 'Raiting', 'Readers', 'Comments'])

In [53]:
# final_df = final_df.append(data_row, ignore_index=True).dropna(axis = 1)

In [54]:
final_df

,Title,Author,Description,Raiting,Readers,Comments


In [262]:
book_links

['https://topliba.com/genres/12090?p=1',
 'https://topliba.com/genres/12090?p=2',
 'https://topliba.com/genres/12090?p=3',
 'https://topliba.com/genres/12090?p=4',
 'https://topliba.com/genres/12090?p=5',
 'https://topliba.com/genres/12090?p=6',
 'https://topliba.com/genres/12090?p=7',
 'https://topliba.com/genres/12090?p=8',
 'https://topliba.com/genres/12090?p=9',
 'https://topliba.com/genres/12090?p=10',
 'https://topliba.com/genres/12090?p=11',
 'https://topliba.com/genres/12090?p=12',
 'https://topliba.com/genres/12090?p=13',
 'https://topliba.com/genres/12090?p=14',
 'https://topliba.com/genres/12090?p=15',
 'https://topliba.com/genres/12090?p=16',
 'https://topliba.com/genres/12090?p=17',
 'https://topliba.com/genres/12090?p=18',
 'https://topliba.com/genres/12090?p=19',
 'https://topliba.com/genres/12090?p=20',
 'https://topliba.com/genres/12090?p=21',
 'https://topliba.com/genres/12090?p=22',
 'https://topliba.com/genres/12090?p=23',
 'https://topliba.com/genres/12090?p=24',
 

In [55]:
for book_link in tqdm_notebook(book_links):
    try:
        data_row = getBookData(book_link)
        final_df = final_df.append(data_row, ignore_index=True)
        time.sleep(0.3)
    except:
        continue

C:\Users\Olga\AppData\Local\Temp\ipykernel_5852\2189728397.py:1: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for book_link in tqdm_notebook(book_links):


  0%|          | 0/1000 [00:00<?, ?it/s]

In [56]:
final_df.head()

,Title,Author,Description,Raiting,Readers,Comments
